<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Generative AI with OpenAI API</h1>
<h1>DALL-E</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg

from IPython.display import Image

import openai
from openai import OpenAI

import os

import requests
import shutil

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.21.0

Compiler    : Clang 15.0.0 (clang-1500.1.0.2.5)
OS          : Darwin
Release     : 23.3.0
Machine     : x86_64
Processor   : i386
CPU cores   : 16
Architecture: 64bit

Git hash: 1f87e80538ad172ebadf16b8ffe7f1e01f363ed6

matplotlib: 3.8.2
watermark : 2.4.3
requests  : 2.31.0
tqdm      : 4.66.1
numpy     : 1.26.4
openai    : 1.12.0
json      : 2.0.9



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

Get the API key

In [4]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

The directory where we will be saving our images

In [5]:
image_dir = "./images/"

# Generate an image based on a Prompt

In [ ]:
prompt = "A wimsical image of a toy duck with an eye patch and hat floating in the clouds"

Make the request

In [ ]:
%%time
response = client.images.generate(
    prompt=prompt,
    n=1,
    size="1024x1024",
    response_format="url",
)

The response includes the URL of the image so that we can download it

In [ ]:
print(response)

Extract the URL from the response

In [ ]:
image_url = response.data[0].url

Define a utility function to help us download the image file

In [ ]:
def download_file(url, filename):
    with requests.get(url, stream=True) as r:
        total_length = int(r.headers.get("Content-Length"))
        with tqdm.wrapattr(r.raw, "read", total=total_length, desc=filename) as raw:
            with open(filename, 'wb')as output:
                shutil.copyfileobj(raw, output)

Download the image

In [ ]:
image_filename = os.path.join(image_dir, 'ducky.png')
download_file(image_url, image_filename)

And display it

In [ ]:
Image(filename=image_filename)

# Generate Prompt

In [ ]:
%%time
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": """You're a brilliant graphic designer. 
        Generate an precise prompt for DALL-E that has at most 1000 characters."""
        },
        {"role": "user", "content": prompt},
    ]
)

In [ ]:
prompt

In [ ]:
print(response.choices[0].message.content)

In [ ]:
expanded_prompt = response.choices[0].message.content

In [ ]:
len(expanded_prompt)

In [ ]:
%%time
response = client.images.generate(
    prompt=expanded_prompt[:1000],
    n=3,
    size="1024x1024",
    response_format="url",
)

In [ ]:
filenames = []

for i in range(3):
    expanded_image_url = response.data[i].url

    expanded_image_filename = os.path.join(image_dir, 'expanded_ducky_%u.png' % i)
    download_file(expanded_image_url, expanded_image_filename)
    filenames.append(expanded_image_filename)

In [ ]:
Image(filenames[0])

In [ ]:
Image(filenames[1])

In [ ]:
Image(filenames[2])

<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>